In [15]:
import csv
from nltk.tokenize.moses import MosesTokenizer
#from nltk.stem import WordNetLemmatizer
import pprint
from collections import Counter, defaultdict
import _pickle as cPickle

reader = csv.DictReader(open('trainset.csv', 'rt', encoding='utf-8'))
pp = pprint.PrettyPrinter(indent=4)
#wordnet_lemmatizer = WordNetLemmatizer()
tknzr = MosesTokenizer()
references = []
vocabulary = Counter()
ovocabulary = defaultdict(lambda: [])

first_words = defaultdict(lambda: 0)
feature_vocabulary = defaultdict(lambda: defaultdict(lambda: Counter()))
for row in reader:
    i1 = row['mr']
    i2 = row['ref']
    
    
    #lem_toks = [wordnet_lemmatizer.lemmatize(tok) for tok in tokens]
    name = ''
    food = 'adsddass'
    keywords = i1.split(',')
    for keyword in keywords:
        k_idx = keyword.find('[')
        k = keyword[:k_idx].strip()
        value = keyword[k_idx + 1: keyword.find(']')]
        ovocabulary[k].append(value)
        #feature_vocabulary[k][value].update(lem_toks)
        if k == 'name':
            name = value
        if k == 'food':
            food = value
    
    tokens = tknzr.tokenize(i2.replace(name, 'XNAMEX').replace(food, 'XFOODX'))
    first_words[tokens[0]] += 1
    #references.append(lem_toks)
    vocabulary.update(tokens)

    
sorted_vocab = sorted(ovocabulary.items(), key=lambda x: len(x[1]), reverse=True)
print(ovocabulary.keys())
print({k: len(v) for k,v in ovocabulary.items()})
print({k: len(set(v)) for k,v in ovocabulary.items()})    


idx_counter = 0
first_words_vocab = {}
for x, freq in first_words.items():
    if freq > 40:
        first_words_vocab[x] = idx_counter
        idx_counter += 1

cPickle.dump(first_words_vocab, open('fw_vocab.pkl', 'wb'))
print(sorted(first_words_vocab.items(), key= lambda x: x[1], reverse=True))     


dict_keys(['name', 'eatType', 'priceRange', 'customer rating', 'near', 'food', 'area', 'familyFriendly'])
{'name': 42061, 'eatType': 20111, 'priceRange': 29127, 'customer rating': 28090, 'near': 20546, 'food': 35126, 'area': 24716, 'familyFriendly': 26295}
{'name': 34, 'eatType': 3, 'priceRange': 6, 'customer rating': 6, 'near': 19, 'food': 7, 'area': 2, 'familyFriendly': 2}
[('the', 38), ('Next', 37), ('coffee', 36), ('Situated', 35), ('Family', 34), ('Yes', 33), ('Come', 32), ('Cheap', 31), ('This', 30), ('Customers', 29), ('Although', 28), ('there', 27), ('You', 26), ('Kid', 25), ('By', 24), ('Rated', 23), ('Riverside', 22), ('Low', 21), ('Offering', 20), ('High', 19), ('Highly', 18), ('Moderately', 17), ('At', 16), ('If', 15), ('Serving', 14), ('With', 13), ('Children', 12), ('The', 11), ('XFOODX', 10), ('On', 9), ('There', 8), ('Near', 7), ('An', 6), ('A', 5), ('In', 4), ('For', 3), ('Located', 2), ('Close', 1), ('XNAMEX', 0)]


In [53]:
reduced_vocab = [(v, f) for v,f in vocabulary.items() if f > 25]
print(len(reduced_vocab))
print(sorted(reduced_vocab, key=lambda x: x[1], reverse=True))

585
[('.', 59079), ('is', 52713), ('a', 41469), ('The', 30884), ('the', 26694), (',', 22728), ('food', 20287), ('in', 18698), ('near', 18191), ('of', 17770), ('and', 17518), ('rating', 16388), ('friendly', 14988), ('customer', 14521), ('restaurant', 14163), ('price', 13350), ('It', 12965), ('located', 12367), ('5', 12056), ('riverside', 10915), ('ha', 10367), ('coffee', 10304), ('shop', 10023), ('range', 9586), ('out', 9090), ('with', 8867), ('city', 8317), ('family', 7420), ('centre', 7381), ('£', 6873), ('serf', 6750), ('that', 6710), ('high', 6664), ('pub', 6178), ('an', 6017), ('Café', 5598), ('child', 5392), ('average', 5347), ('not', 5344), ('low', 5286), ('Indian', 4907), ('area', 4855), ('Italian', 4825), ('than', 4739), ('Japanese', 4651), ('priced', 4560), ('kid', 4501), ('it', 4485), ('French', 4448), ('by', 4388), ('called', 4351), ('rated', 3979), ('cheap', 3918), ('Rice', 3902), ('Boat', 3888), ('for', 3634), ('to', 3606), ('family-friendly', 3512), ('at', 3412), ('fast',

In [57]:
feature_domain = {k: set(v) for k,v in ovocabulary.items()}
print(feature_domain)

{'name': {'Bibimbap House', 'The Cricketers', 'Browns Cambridge', 'Fitzbillies', 'The Golden Curry', 'The Rice Boat', 'The Punter', 'Loch Fyne', 'Cocum', 'The Olive Grove', 'The Vaults', 'The Cambridge Blue', 'The Plough', 'Cotto', 'The Eagle', 'Taste of Cambridge', 'The Dumpling Tree', 'Zizzi', 'The Mill', 'The Golden Palace', 'Strada', 'Travellers Rest Beefeater', 'The Waterman', 'Aromi', 'Alimentum', 'Giraffe', 'Midsummer House', 'Clowns', 'Green Man', 'The Twenty Two', 'The Wrestlers', 'Blue Spice', 'Wildwood', 'The Phoenix'}, 'eatType': {'restaurant', 'pub', 'coffee shop'}, 'priceRange': {'cheap', 'less than £20', 'moderate', '£20-25', 'high', 'more than £30'}, 'customer rating': {'3 out of 5', '5 out of 5', '1 out of 5', 'high', 'average', 'low'}, 'near': {'The Rice Boat', 'The Six Bells', 'Yippee Noodle Bar', 'The Sorrento', 'Café Sicilia', 'Rainbow Vegetarian Café', 'Avalon', 'Café Rouge', 'Express by Holiday Inn', 'Café Brazil', 'Café Adriatic', 'Clare Hall', 'The Bakers', 'Th

In [67]:
feature_vocabulary['familyFriendly']['no'].most_common(10)

[('.', 14292),
 ('is', 13462),
 ('a', 8911),
 ('The', 6799),
 ('the', 6279),
 (',', 5486),
 ('not', 5134),
 ('in', 4716),
 ('and', 4558),
 ('food', 4548)]

In [102]:
feature_vocabulary_rel  = defaultdict(lambda: defaultdict(lambda: Counter()))

for feautre, values in feature_vocabulary.items():
    for value, tokens in values.items():
        for token, freq in tokens.most_common(25):
            feature_vocabulary_rel[feautre][value][token] = freq/vocabulary[token]
    


In [104]:
feature_vocabulary_rel['priceRange']['more than £30'].most_common(100)

[('30', 0.996594778660613),
 ('more', 0.9713518352730528),
 ('than', 0.45326018147288455),
 ('£', 0.32082060235704934),
 ('child', 0.269473293768546),
 ('range', 0.18057583976632588),
 ('price', 0.17340823970037453),
 ('coffee', 0.13480201863354038),
 ('and', 0.12695513186436808),
 (',', 0.12565997888067582),
 ('a', 0.1224288022378162),
 ('friendly', 0.11949559647718175),
 ('the', 0.11879073949202068),
 ('is', 0.11740936770815548),
 ('.', 0.11669121007464581),
 ('near', 0.11555164641855863),
 ('in', 0.1150390416087282),
 ('It', 0.11407635942923255),
 ('located', 0.11280019406485),
 ('The', 0.1116435694858179),
 ('customer', 0.1073617519454583),
 ('restaurant', 0.10541551931088046),
 ('rating', 0.10263607517695875),
 ('of', 0.10236353404614519),
 ('food', 0.0938532064869128)]